In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import csv

#Import module from folder in another directory
import importlib.util
spec = importlib.util.spec_from_file_location("outliers", "data_preprocessing/outliers.py")
outliers = importlib.util.module_from_spec(spec)
spec.loader.exec_module(outliers)

import sys
sys.path.append('data_preprocessing')
import data_preprocessing.summary as summary
import data_preprocessing.normalize as normalize

import warnings
warnings.filterwarnings("ignore")

from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn import svm

#import xgboost as xgb

%matplotlib inline

rng = np.random.RandomState(31337)

In [5]:
train_data = pd.read_csv('../data/input/ames_train.csv')
test_data = pd.read_csv('../data/input/ames_test.csv')

In [7]:
'''train_data.set_index('Id', inplace=True)
train_data.drop(['PID', 'Latitude', 'Longitude'], axis=1, inplace=True)

test_data.set_index('Id', inplace=True)
test_data.drop(['PID', 'Latitude', 'Longitude'], axis=1, inplace=True)'''

"train_data.set_index('Id', inplace=True)\ntrain_data.drop(['PID', 'Latitude', 'Longitude'], axis=1, inplace=True)\n\ntest_data.set_index('Id', inplace=True)\ntest_data.drop(['PID', 'Latitude', 'Longitude'], axis=1, inplace=True)"

In [8]:
train_data = normalize.fill_in_missing_values(train_data)
train_data = outliers.remove_outliers(train_data)
#train_data = normalize.normalize(train_data)

test_data = normalize.fill_in_missing_values(test_data)
test_data = outliers.remove_outliers(test_data)

In [9]:
train_data = train_data.select_dtypes(include=[np.number])
test_data = test_data.select_dtypes(include=[np.number])

X_train, X_test, y_train, y_test = train_test_split(train_data.loc[:, train_data.columns != 'SalePrice'], 
                                                  train_data['SalePrice'],
                                                  test_size=0.2, random_state=42)
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

X_train.shape

(1844, 37)

In [ ]:

train_size = np.shape(X_train)[0]
test_size = np.shape(X_test)[0]
num_features = np.shape(X_train)[1]
num_hidden = 16 # Number of activation units in the hidden layer


# Neural network with one hidden layer
graph = tf.Graph()
with graph.as_default():

    # Input
    tf_train_dataset = tf.constant(X_train, dtype=tf.float64)
    tf_train_labels = tf.constant(y_train, dtype=tf.float64)
    tf_test_dataset = tf.constant(X_test)

    # Variables
    weights_1 = tf.Variable(tf.truncated_normal([num_features, num_hidden]), dtype=tf.float32, name="layer1_weights")
    weights_1 = tf.cast(weights_1, tf.float64)
    #weights = tf.Variable(tf.truncated_normal([num_features, 1]), name="weights")
    biases_1 = tf.Variable(tf.zeros([num_hidden]), dtype=tf.float32, name="layer1_biases")
    biases_1 = tf.cast(biases_1, tf.float64)
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden,1]), dtype=tf.float32, name="layer2_weights")
    weights_2 = tf.cast(weights_2, tf.float64)
    #weights = tf.Variable(tf.truncated_normal([num_features, 1]), name="weights")
    biases_2 = tf.Variable(tf.zeros([1]), dtype=tf.float32, name="layer2_biases")
    biases_2 = tf.cast(biases_2, tf.float64)
    steps = tf.Variable(0)
    
     # Model
    def model(x, train=False):
        hidden = tf.nn.relu(tf.matmul(x, weights_1) + biases_1)
        return tf.matmul(hidden, weights_2) + biases_2
    
    # Loss Computation
    train_prediction = model(tf_train_dataset)
    loss = 0.5 * tf.reduce_mean(tf.squared_difference(tf_train_labels, train_prediction))
    cost = tf.sqrt(loss)

    # Optimizer
    # Exponential decay of learning rate
    learning_rate = tf.train.exponential_decay(0.06, steps, 5000, 0.70, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost, global_step=steps)

    # Predictions
    test_prediction = model(tf_test_dataset)

    saver = tf.train.Saver()

num_steps = 100001

def accuracy(prediction, labels):
    return 0.5 * np.sqrt(((prediction - labels) ** 2).mean(axis=None))

# Running graph
with tf.Session(graph=graph) as sess:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
      # Run the computations. We tell .run() that we want to run the optimizer,
      # and get the loss value and the training predictions returned as numpy
      # arrays.
      _, c, predictions = sess.run([optimizer, cost, train_prediction])
      if (step % 5000 == 0):
          print('Cost at step %d: %.2f' % (step, c))
          # Calling .eval() on valid_prediction is basically like calling run(), but
          # just to get that one numpy array. Note that it recomputes all its graph
          # dependencies.
          #print('Validation loss: %.2f' % accuracy(valid_prediction.eval(), y_valid))
  t_pred = test_prediction.eval()
  print('Test loss: %.2f' % accuracy(t_pred, y_test))
  save_path = saver.save(sess, "./model/nn-model.ckpt")
  print('Model saved in %s' % (save_path))
  
  print("r2 score: ", r2_score(y_test,t_pred))
    

Initialized
Cost at step 0: 52207416.15
